# Train the model by preparing the dataset

In [ ]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import json
from src import preparedata
from src import modelareaegpdcombined as modelarea
from src import trainarea
import seaborn as sns
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from scipy.stats import expon

print(tf.__version__)
print(tf.config.list_physical_devices(device_type=None))
params = json.load(open("params/paramsegpdcombined.json", "r"))

In [ ]:
# prepare data
dataset = preparedata.readGPDData(params["dataprepinargs"])
dataset.preparedata()

In [ ]:
all_area = np.append(dataset.Y_train[:, 1], dataset.Y_test[:, 1])
all_area = all_area[all_area > 0.0]
np.save("Data/Inventory/all_area.npy", all_area)

In [ ]:
# create random balanced sample
posloc = np.where(dataset.Y_train[:, 0] > 0)[0]
negloc = np.where(dataset.Y_train[:, 0] == 0)[0]
selneg = np.random.choice(negloc, size=posloc.shape[0], replace=False)
sel = np.concatenate([posloc, selneg])
sel.sort()

dataset.Y_train = dataset.Y_train[sel]
dataset.X_train = dataset.X_train[sel]

In [ ]:
# prepare model
landslidehazard = modelarea.lhmodel(params["modelparam"])
landslidehazard.preparemodel()

In [ ]:
# train the model
version = "20230823V1egpdcombined"
trainarea.trainmodel(
    landslidehazard.model, dataset.X_train, [dataset.Y_train[:,0],dataset.Y_train[:,1]], params["trainparam"]
)
# landslidehazard.model.load_weights(f"checkpoints/")
landslidehazard.model.save_weights(f"savedweights/final_model{version}.h5")

In [ ]:
# predict from trained model
version = "20230823V1egpdcombined"
landslidehazard.model.load_weights(f"savedweights/final_model{version}.h5")

In [ ]:
ypred = landslidehazard.model.predict(
    dataset.X_test.astype(np.float32), batch_size=2048
)
yprob = ypred[0]
ypred = ypred[1][dataset.Y_test[:, 1] > 0.1]
yarea = dataset.Y_test[:, 1][dataset.Y_test[:, 1] > 0.1]

In [ ]:


def eGPD_cdf(y, k, sig, xi):
    return (1 - (1 + xi * y / sig) ** (-1 / xi)) ** k


def eGPD_ppf(p, sig, k=0.8118067, xi=0.4919825):
    """
    Calculate the quantile for a Extended Generalized Pareto Distribution.

    Parameters:
        p (float): Probability level (0 < p < 1).
        xi (float): Shape parameter.
        sig (float): Scale parameter.
        k (float): Parameter k.

    Returns:
        float: Quantile value.
    """
    return (sig / xi) * (((1 - (p ** (1 / k))) ** (-xi)) - 1)


def MSE(sig, x, k=0.8118067, xi=0.4919825, area=1):
    xi = xi
    sigma = (tf.nn.relu(sig).numpy() + 0.2) * area
    kappa = k
    if xi <= 0:
        return 1e10

    dat = x[x > 0]

    exp_dat = eGPD_cdf(dat, k=kappa, sig=sigma[x > 0], xi=xi)
    exp_dat = expon.ppf(exp_dat)

    p_min = 0
    n_p = len(exp_dat) * (1 - p_min)
    ps = p_min + np.arange(1, int(n_p) + 1) / (n_p + 1) * (1 - p_min)
    mse = np.mean((np.quantile(exp_dat, ps) - expon.ppf(ps)) ** 2)

    return mse


def getquantiles(sig, x, k=0.8118067, xi=0.4919825, area=1):
    xi = xi
    sigma = (sig + 0.2) * area
    kappa = k
    if xi <= 0:
        return 1e10

    dat = x[x > 0]

    exp_dat = eGPD_cdf(dat, k=kappa, sig=sigma[x > 0], xi=xi)
    exp_dat = expon.ppf(exp_dat)

    p_min = 0
    n_p = len(exp_dat) * (1 - p_min)
    ps = p_min + np.arange(1, int(n_p) + 1) / (n_p + 1) * (1 - p_min)
    return np.quantile(exp_dat, ps), expon.ppf(ps)

In [ ]:
sns.kdeplot(eGPD_ppf((1 - 1 / 10), ypred + 0.2, k=0.8118067, xi=0.4919825))

In [ ]:
q1, q2 = getquantiles(ypred, yarea)
plt.scatter(
    q2,
    q1,
)
plt.xlabel("Theoritical Quantiles")
plt.ylabel("Observed Quantiles")
plt.axline([0, 0], [1, 1])
plt.xlim(0, 10)
plt.ylim(0, 10)
plt.axis("square")
plt.savefig("Data/Plots/quantilesv5.pdf", dpi=300)

In [ ]:
fpr, tpr, thresholds = sklearn.metrics.roc_curve(dataset.Y_test[:, 0], yprob)
auc = sklearn.metrics.auc(fpr, tpr)
acc = sklearn.metrics.balanced_accuracy_score(dataset.Y_test[:, 0], yprob > 0.5)

plt.plot(
    fpr,
    tpr,
    label=f"auc ={ round(auc,2)}",
)
ax = plt.plot([0, 1], [0, 1], color="navy", linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve Landslide Classification")
# plt.text(0.38, 0.11, "Balanced Accuracy=%0.2f" % acc)
plt.legend(loc="lower right", prop={"size": 10})
plt.axis("square")
plt.savefig("Data/Plots/rocv5.pdf", dpi=300)

In [ ]:
sklearn.metrics.balanced_accuracy_score(dataset.Y_test[:, 0], yprob > 0.50)